In [24]:
import json
import pandas as pd

with open('../data/mediastack/india-news.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['data'])


In [3]:
df['description'] = df['description'].fillna('')
df['description'] = df['description'].astype(pd.StringDtype())
descriptions = df['description'].astype(str).tolist()

In [4]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")
df['sentiment'] = sentiment_pipeline(descriptions)

/home/vrushank/newsAggregator/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/vrushank/newsAggregator/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# break down the sentiment column into columns label and score
df = pd.concat([df.drop(['sentiment'], axis=1), df['sentiment'].apply(pd.Series)], axis=1)

# get all rows with Positive sentiment
positive = df[df['label'] == 'POSITIVE']

40


In [10]:
# drop the country and language columns
print(positive.drop(['country', 'language'], axis=1).columns.tolist())

['author', 'title', 'description', 'url', 'source', 'image', 'category', 'published_at', 'label', 'score']


In [11]:
# find the different categories and total count for all positive rows
print(positive['category'].value_counts())
print(positive['category'].value_counts().sum())

category
general       26
sports        11
business       2
technology     1
Name: count, dtype: int64
40


In [22]:
# print all the titles of the positive rows in a list with the category
def print_titles(df):
    for _, row in df.iterrows():
        print(row['description'])
        print(row['category'])
        print(row['score'])
        print('\n\n')

In [15]:
# write this df as a json file
df.to_json('../data/mediastack/sentiments.json', orient='records')

In [23]:
negatives = df[df['label'] == 'NEGATIVE']
# print(negatives['category'].value_counts())

print_titles(negatives)

US News Live Updates Today September 3, 2024: WWE superstar Bronson Reed ‘medically unable to compete’ in RAW&nbsp;&nbsp;Hindustan TimesView Full coverage on Google News
general
0.9945023059844971



Man Drugged Wife, Invited 72 Strangers To Rape Her Nearly 100 Times Over 10 Years&nbsp;&nbsp;News18French woman drugged by husband, raped by 51 men asks for public trial: ‘Shame must change sides'&nbsp;&nbsp;Hindustan TimesTearful trauma at Frenchman’s trial over recruiting dozens to rape wife for decade&nbsp;&nbsp;DAWN.comShocking Betrayal: Husband in France allegedly drugged wife, allowed 50 men to rape her nearly 100 times for 10 years&nbsp;&nbsp;The Economic TimesFrench man accused of recruiting over 50 strangers to rape wife after drugging her&nbsp;&nbsp;India Today
general
0.9936537742614746



Brazil’s Supreme Court bans X: Why the escalating row rings familiar in India&nbsp;&nbsp;The Indian ExpressStarlink emerges as fresh battleground between Elon Musk, Brazil&nbsp;&nbsp;The Econo